In [1]:
import numpy as np
import pandas as pd

In [6]:
aep = pd.read_csv('./data/AEP_hourly.csv')
comed = pd.read_csv('./data/COMED_hourly.csv')
dayton = pd.read_csv('./data/DAYTON_hourly.csv')
deok = pd.read_csv('./data/DEOK_hourly.csv')
dom = pd.read_csv('./data/DOM_hourly.csv')
duq = pd.read_csv('./data/DUQ_hourly.csv')
ekpc = pd.read_csv('./data/EKPC_hourly.csv')
fe = pd.read_csv('./data/FE_hourly.csv')
ni = pd.read_csv('./data/NI_hourly.csv')
pjm = pd.read_csv('./data/PJM_Load_hourly.csv')
pjme = pd.read_csv('./data/PJME_hourly.csv')
pjmw = pd.read_csv('./data/PJMW_hourly.csv')